# Use Azure Map Service
Get informations about every italian city

In [1]:
import aiohttp
import pandas as pd
import datetime
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from IPython.display import Image, display
from utility import get_folder_path, define_path, read_dataset, save_dataset
from yaml_utility import read_yaml_file

## Analyze and get Latitude and Longitude for every Comune in Italy

In [2]:
#Get italian city data
folder_path = get_folder_path('../../Dataset')
file_path = define_path(folder_path, 'comuni_italiani.csv','')
df = read_dataset(file_path,csv_sep=';',encoding='utf-8')

PATH: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Dataset, force creation: False

 Dataset location: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Dataset/comuni_italiani.csv 

Found csv in the path
Dataset loaded


In [3]:
comuni_columns = ['Codice catastale','Codice comune','Nome Comune Internazionale','Provincia','Regione']
df = df[comuni_columns]
df = df.rename(columns={'Nome Comune Internazionale': 'Comune', 'Codice Catastale':'CodiceCatastale','Codice comune':'CodiceComune'})
df.head(10)

,Codice catastale,CodiceComune,Comune,Provincia,Regione
0,A074,1001,Agliè,Torino,Piemonte
1,A109,1002,Airasca,Torino,Piemonte
2,A117,1003,Ala di Stura,Torino,Piemonte
3,A157,1004,Albiano d'Ivrea,Torino,Piemonte
4,A218,1006,Almese,Torino,Piemonte
5,A221,1007,Alpette,Torino,Piemonte
6,A222,1008,Alpignano,Torino,Piemonte
7,A275,1009,Andezeno,Torino,Piemonte
8,A282,1010,Andrate,Torino,Piemonte
9,A295,1011,Angrogna,Torino,Piemonte


#### Import configurations file for Azure Map Service

In [21]:
config_base_path = get_folder_path('./')
configurations = read_yaml_file(config_base_path,'azuremaps_config.yml')

client_id = configurations['client_id']
subscription_id = configurations['subscription_id']
subscription_key = configurations['subscription_key']
language = configurations['language']
country = configurations['country']

PATH: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Code/utility, force creation: False
File path yaml checked to read/write from: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Code/utility/azuremaps_config.yml
File succesfully readed


In [5]:
#Documentation: https://docs.microsoft.com/en-us/rest/api/maps/search/getsearchaddress
service_uri = "https://atlas.microsoft.com/search/address/json?subscription-key={}&api-version=1.0&query={}&countrySet={}&language={}"

In [10]:
latitude = []
longitude = []
comune_list = []
provincia_list = []
regione_list = []

In [11]:
session = aiohttp.ClientSession()
errors = []

for i, el in enumerate(tqdm_notebook(df['CodiceComune'].tolist())):
    
    comune = df.loc[i,'Comune']
    provincia = df.loc[i,'Provincia']
    regione = df.loc[i,'Regione']
    
    query = str(comune) + ', ' + str(provincia) + ', ' + str(regione) 
    request = service_uri.format(subscription_key,query,country,language)
    
    try:
        response = await(await session.get(request)).json()
    
        #response['results'][0]['address']['countrySecondarySubdivision']
        comune_list.append(comune)
        provincia_list.append(provincia)
        regione_list.append(regione)
        latitude.append(response['results'][0]['position']['lat'])
        longitude.append(response['results'][0]['position']['lon'])

    except Exception as message:
        print(f"Impossibile to get information for element {i} about: {comune}, {provincia}, {regione} because: {message}")
        errors.append([i,el,comune,provincia,regione])
        continue

await session.close()
print(f"Download completed with {len(errors)} errors. Please check the errors list to see informations")


/Users/jeydi/opt/anaconda3/envs/develop/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


Impossibile to get information for element 2784 about: Cortaccia sulla strada del vino/Kurtatsch an der Weinstraße, Bolzano/Bozen, Trentino-Alto Adige/Südtirol because: list index out of range
Impossibile to get information for element 2785 about: Cortina sulla strada del vino/Kurtinig an der Weinstraße, Bolzano/Bozen, Trentino-Alto Adige/Südtirol because: list index out of range

Download completed with 2 errors. Please check the errors list to see informations


Append to original dataframe the data you have obtaines from the service

In [17]:
#Remove errors from original dataframe if there are into the previous procedure
df_no_errors = df.copy()

if errors != []:
    for i,e in enumerate(errors):
        codice = e[1]
        # Get names of indexes for which column Age has value 30
        indexNames = df_no_errors[ df_no_errors['CodiceComune'] == codice ].index
 
        # Delete these row indexes from dataFrame
        df_no_errors.drop(indexNames , inplace=True)
        print(f"Removed errors for CodiceComune {codice} in position: {indexNames}")

Removed errors for CodiceComune 21024 in position: Int64Index([2784], dtype='int64')
Removed errors for CodiceComune 21025 in position: Int64Index([2785], dtype='int64')


In [18]:
#Create columns into the result_dataframe
df_no_errors['Latitude'] = latitude
df_no_errors['Longitude'] = longitude
df_no_errors.shape

(7902, 7)

Save the result dataframe to disk

In [23]:
output_path = define_path(folder_path, 'comuni_italiani_edited.csv','')
save_dataset(output_path, df_no_errors,csv_sep=";")


 Dataset location: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Dataset/comuni_italiani_edited.csv 

File saved succesfully to: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Dataset/comuni_italiani_edited.csv


True

## Obtain Latitude and Longitude for every MBE Store with usefull informations

Import original mbe dataset

In [26]:
mbe_dataset_path = define_path(folder_path, '1_etlStep3_finalTable_temp.csv','')
df_mbe = read_dataset(mbe_dataset_path, csv_sep=',', encoding='utf-8')
print(f"Shape: {df_mbe.shape}")
df_mbe.head(10)


 Dataset location: /Users/jeydi/Dropbox/Progetti/LAVORO/DataLab/Rework/DatalabV1_Rework/Dataset/1_etlStep3_finalTable_temp.csv 

Found csv in the path
Dataset loaded
Shape: (117847, 94)


/Users/jeydi/opt/anaconda3/envs/develop/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,STOREBASE,ANNO,MESE,FRANCHISEENO,FRANCHISEENAME,PREVIOUSFRANCHISEE,CLASSEDFS,SEGMENTOGS,REGIONESTORE,ASSISTENZAGOLD,...,TOTALEXTRATOTALGROSSCOST,TOTALEXTRATOTALGROSSPRICE,MAXCOURIERSERVICE,COUNTMAXCOURIERSERVICE,MINCOURIERSERVICE,COUNTMINCOURIERSERVICE,MAXSHIPSERVICE,COUNTMAXSHIPSERVICE,MINSHIPSERVICE,COUNTMINSHIPSERVICE
0,IT0734,2013,9,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,20.55,623.079987,Standard,12.0,Saver,2.0,MBE Standard,19.0,MBE Express,2.0
1,IT0734,2013,10,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,13.12,279.279999,ExtraLarge,9.0,Saver,2.0,MBE Standard,15.0,MBE Express,2.0
2,IT0734,2013,11,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,23.18,466.970013,Standard,8.0,Expedited,1.0,MBE Standard,15.0,MBE Standard,15.0
3,IT0734,2013,12,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,3.05,0.000000,Standard,9.0,Saver,2.0,MBE Standard,22.0,MBE Express,2.0
4,IT0734,2014,1,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,9.15,75.000000,ExtraLarge,26.0,Saver,1.0,MBE Express,33.0,MBE Standard,1.0
5,IT0734,2014,2,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,15.12,2.450000,ExtraLarge,68.0,Saver,1.0,MBE Express,75.0,MBE Standard,1.0
6,IT0734,2014,3,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,3.42,2.310000,ExtraLarge,20.0,Golden,2.0,MBE Standard,27.0,MBE Express P+,2.0
7,IT0734,2014,4,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,17.08,0.000000,Standard,4.0,Expedited,1.0,MBE Standard,7.0,MBE Express,2.0
8,IT0734,2014,5,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,8.54,0.000000,ExtraLarge,10.0,Golden,1.0,MBE Express P+,19.0,MBE Standard,1.0
9,IT0734,2014,6,752171602,IT0734,NaN,B,0.0,Puglia,1.0,...,0.00,0.000000,ExtraLarge,11.0,Saver,2.0,MBE Express,11.0,MBE Standard,2.0


Warning: we don't have the informations about the roads

Get only usefull informations for the search

Do the search

Save results